In [300]:
#importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
from sklearn.impute import SimpleImputer

In [301]:
#importing the incident dataset for analysis
incident=pd.read_csv("./Data/incident_event_log.csv")
incident.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,u_priority_confirmation,notify,problem_id,rfc,vendor,caused_by,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,0,0,0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,0,0,2,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,0,0,3,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,0,0,4,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,0,0,0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,False,Do Not Notify,?,?,?,?,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [302]:
#Analysing the shape of the dataframe
incident.shape

(141712, 36)

In [303]:
#Calculating NA's in each column
incident.isnull().sum(axis=0) 

number                     0
incident_state             0
active                     0
reassignment_count         0
reopen_count               0
sys_mod_count              0
made_sla                   0
caller_id                  0
opened_by                  0
opened_at                  0
sys_created_by             0
sys_created_at             0
sys_updated_by             0
sys_updated_at             0
contact_type               0
location                   0
category                   0
subcategory                0
u_symptom                  0
cmdb_ci                    0
impact                     0
urgency                    0
priority                   0
assignment_group           0
assigned_to                0
knowledge                  0
u_priority_confirmation    0
notify                     0
problem_id                 0
rfc                        0
vendor                     0
caused_by                  0
closed_code                0
resolved_by                0
resolved_at   

In [304]:
#Checking for the datatypes present in the dataframe
incident.dtypes

number                     object
incident_state             object
active                       bool
reassignment_count          int64
reopen_count                int64
sys_mod_count               int64
made_sla                     bool
caller_id                  object
opened_by                  object
opened_at                  object
sys_created_by             object
sys_created_at             object
sys_updated_by             object
sys_updated_at             object
contact_type               object
location                   object
category                   object
subcategory                object
u_symptom                  object
cmdb_ci                    object
impact                     object
urgency                    object
priority                   object
assignment_group           object
assigned_to                object
knowledge                    bool
u_priority_confirmation      bool
notify                     object
problem_id                 object
rfc           

In [305]:
#Estimating the range of values in the dataset
incident.describe()

,reassignment_count,reopen_count,sys_mod_count
count,141712.000000,141712.000000,141712.000000
mean,1.104197,0.021918,5.080946
std,1.734673,0.207302,7.680652
min,0.000000,0.000000,0.000000
25%,0.000000,0.000000,1.000000
50%,1.000000,0.000000,3.000000
75%,1.000000,0.000000,6.000000
max,27.000000,8.000000,129.000000


In [306]:
#df.drop(['cmdb_ci', 'problem_id', 'rfc', 'vendor','caused_by','caller_id','opened_by','sys_created_by','sys_updated_by','location','resolved_by','notify','assigned_to','assignment_group'],axis = 1,inplace=True)

In [307]:
 #Replacing ? in the dataframe
incident.replace('?', np.NaN, inplace = True)
incident.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 141712 entries, 0 to 141711
Data columns (total 36 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   number                   141712 non-null  object
 1   incident_state           141712 non-null  object
 2   active                   141712 non-null  bool  
 3   reassignment_count       141712 non-null  int64 
 4   reopen_count             141712 non-null  int64 
 5   sys_mod_count            141712 non-null  int64 
 6   made_sla                 141712 non-null  bool  
 7   caller_id                141683 non-null  object
 8   opened_by                136877 non-null  object
 9   opened_at                141712 non-null  object
 10  sys_created_by           88636 non-null   object
 11  sys_created_at           88636 non-null   object
 12  sys_updated_by           141712 non-null  object
 13  sys_updated_at           141712 non-null  object
 14  contact_type        

In [308]:
#Generating a heatmap to explore the columns which have most NA values
#fig, ax = plt.subplots(figsize=(10,10))  
#sns.heatmap(incident.isnull(), cmap='viridis',ax=ax) 

In [309]:
#Removing the columns which had NA values the most
incident_processed = incident.drop(['cmdb_ci', 'problem_id', 'rfc', 'vendor', 'caused_by'],axis=1) 

In [310]:
#remove outliers
incident_processed = incident_processed[incident_processed['incident_state'] != '-100' ]

In [311]:
reass_count_labels = [1,2,3,4,5]
incident_processed['reassignment_count'] = pd.cut(incident_processed['reassignment_count'],bins = 5, labels = reass_count_labels)

In [312]:
#Replacing with mean for numerical columns
df_imputer=SimpleImputer(missing_values=np.NaN, strategy='mean', verbose=0)
df_imputer=df_imputer.fit(incident_processed.loc[:,['reopen_count', 'sys_mod_count']])# columns with missing values
incident_processed.loc[:,['reopen_count', 'sys_mod_count']]=df_imputer.transform(incident_processed.loc[:,['reopen_count', 'sys_mod_count']])

In [313]:
# replace subtypes of Awaiting incident state into main type of Awaiting
incident_processed['incident_state'].replace(['Awaiting User Info','Awaiting Problem','Awaiting Vendor','Awaiting Evidence'],'Awaiting',inplace = True)

In [314]:
incident_processed['incident_state'].unique()

array(['New', 'Resolved', 'Closed', 'Active', 'Awaiting'], dtype=object)

In [315]:
#Replacing with mode for categorical columns
for column in ['caller_id', 'opened_by', 'sys_created_by','sys_created_at','location','category','subcategory','u_symptom','assignment_group','assigned_to','closed_code','resolved_by','resolved_at']:
    incident_processed[column].fillna(incident_processed[column].mode()[0], inplace=True) 

In [316]:
incident_processed.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,New,True,1,0.0,0.0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,3 - Moderate,Group 56,Resolver 17,True,False,Do Not Notify,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,Resolved,True,1,0.0,2.0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,3 - Moderate,Group 56,Resolver 17,True,False,Do Not Notify,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,Resolved,True,1,0.0,3.0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,3 - Moderate,Group 56,Resolver 17,True,False,Do Not Notify,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,Closed,False,1,0.0,4.0,True,Caller 2403,Opened by 8,29/2/2016 01:16,...,3 - Moderate,Group 56,Resolver 17,True,False,Do Not Notify,code 5,Resolved by 149,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,New,True,1,0.0,0.0,True,Caller 2403,Opened by 397,29/2/2016 04:40,...,3 - Moderate,Group 70,Resolver 89,True,False,Do Not Notify,code 5,Resolved by 81,1/3/2016 09:52,6/3/2016 10:00


In [317]:
incident_processed.columns

Index(['number', 'incident_state', 'active', 'reassignment_count',
       'reopen_count', 'sys_mod_count', 'made_sla', 'caller_id', 'opened_by',
       'opened_at', 'sys_created_by', 'sys_created_at', 'sys_updated_by',
       'sys_updated_at', 'contact_type', 'location', 'category', 'subcategory',
       'u_symptom', 'impact', 'urgency', 'priority', 'assignment_group',
       'assigned_to', 'knowledge', 'u_priority_confirmation', 'notify',
       'closed_code', 'resolved_by', 'resolved_at', 'closed_at'],
      dtype='object')

In [318]:
# Applying labelencoder to transform categorical value to numerical value
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
incident_processed[['incident_state', 'active', 'reassignment_count','reopen_count', 'sys_mod_count', 'made_sla', 'caller_id',        'opened_by','sys_created_by','sys_updated_by','contact_type', 'location', 'category', 'subcategory','u_symptom', 'impact',            'urgency', 'priority', 'assignment_group','assigned_to', 'knowledge', 'u_priority_confirmation', 'notify','closed_code',              'resolved_by']] = incident_processed[['incident_state', 'active', 'reassignment_count','reopen_count', 'sys_mod_count', 'made_sla', 'caller_id',        'opened_by','sys_created_by','sys_updated_by','contact_type', 'location', 'category', 'subcategory','u_symptom', 'impact',            'urgency', 'priority', 'assignment_group','assigned_to', 'knowledge', 'u_priority_confirmation', 'notify','closed_code',              'resolved_by']].apply(le.fit_transform) 
#df['number'] = incident_processed['number']

In [319]:
incident_processed.head(8)

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,priority,assignment_group,assigned_to,knowledge,u_priority_confirmation,notify,closed_code,resolved_by,resolved_at,closed_at
0,INC0000045,3,1,0,0,0,1,1462,200,29/2/2016 01:16,...,2,47,73,1,0,0,12,51,29/2/2016 11:29,5/3/2016 12:00
1,INC0000045,4,1,0,0,2,1,1462,200,29/2/2016 01:16,...,2,47,73,1,0,0,12,51,29/2/2016 11:29,5/3/2016 12:00
2,INC0000045,4,1,0,0,3,1,1462,200,29/2/2016 01:16,...,2,47,73,1,0,0,12,51,29/2/2016 11:29,5/3/2016 12:00
3,INC0000045,2,0,0,0,4,1,1462,200,29/2/2016 01:16,...,2,47,73,1,0,0,12,51,29/2/2016 11:29,5/3/2016 12:00
4,INC0000047,3,1,0,0,0,1,1462,120,29/2/2016 04:40,...,2,63,223,1,0,0,12,196,1/3/2016 09:52,6/3/2016 10:00
5,INC0000047,0,1,0,0,1,1,1462,120,29/2/2016 04:40,...,2,15,167,1,0,0,12,196,1/3/2016 09:52,6/3/2016 10:00
6,INC0000047,0,1,0,0,2,1,1462,120,29/2/2016 04:40,...,2,15,167,1,0,0,12,196,1/3/2016 09:52,6/3/2016 10:00
7,INC0000047,0,1,0,0,3,1,1462,120,29/2/2016 04:40,...,2,15,167,1,0,0,12,196,1/3/2016 09:52,6/3/2016 10:00


In [320]:
incident_processed.columns

Index(['number', 'incident_state', 'active', 'reassignment_count',
       'reopen_count', 'sys_mod_count', 'made_sla', 'caller_id', 'opened_by',
       'opened_at', 'sys_created_by', 'sys_created_at', 'sys_updated_by',
       'sys_updated_at', 'contact_type', 'location', 'category', 'subcategory',
       'u_symptom', 'impact', 'urgency', 'priority', 'assignment_group',
       'assigned_to', 'knowledge', 'u_priority_confirmation', 'notify',
       'closed_code', 'resolved_by', 'resolved_at', 'closed_at'],
      dtype='object')

In [321]:
incident_processed['reassignment_count'].unique()

array([0, 1, 2, 3, 4], dtype=int64)

In [322]:
incident_processed[['priority','incident_state']].groupby('incident_state').agg(['count'])

,priority
,count
incident_state,
0,38716
1,15848
2,24985
3,36407
4,25751


In [323]:
# df['active'] = df['active'].apply(lambda x : 1 if x == 'True' else 0)
# df['made_sla'] = df['made_sla'].apply(lambda x : 1 if x == 'True' else 0)
# df['knowledge'] = df['knowledge'].apply(lambda x : 1 if x == 'True' else 0)
# df['u_priority_confirmation'] = df['u_priority_confirmation'].apply(lambda x : 1 if x == 'True' else 0)
# df['priority'] = df['priority'].apply(lambda x : 1 if '1' in x else 2 if '2' in x else 3 if '3' in x else 4)
# df['impact'] = df['impact'].apply(lambda x : 1 if '1' in x else 2 if '2' in x else 3 if '3' in x else 4)
# df['urgency'] = df['urgency'].apply(lambda x : 1 if '1' in x else 2 if '2' in x else 3 if '3' in x else 4)
# reass_count_labels = [1,2,3,4,5]
# df['reassignment_count'] = pd.cut(df['reassignment_count'],bins = 5, labels = reass_count_labels)
# df = pd.get_dummies(df,columns = ['incident_state'])

In [324]:
#Convert date time string values into pandas datetime values
incident_processed['closed_at'] = pd.to_datetime(incident_processed['closed_at'])
incident_processed['opened_at'] = pd.to_datetime(incident_processed['opened_at'])
incident_processed['resolved_at'] = pd.to_datetime(incident_processed['resolved_at'])
incident_processed['sys_updated_at'] = pd.to_datetime(incident_processed['sys_updated_at'])


In [325]:
#calculate features like lifetime = time the incident was active, action_at = time taken when first action was undertaken
incident_processed['lifetime'] = incident_processed['closed_at'] - incident_processed['opened_at']
incident_processed['action_at'] = incident_processed['sys_updated_at'] - incident_processed['opened_at']

In [326]:
#extract only the day values from datetime value for features
incident_processed['lifetime'] = incident_processed['lifetime'].dt.days
incident_processed['action_at'] = incident_processed['action_at'].dt.days

In [327]:
#check number of records having close time before open time (which is wrong : It is because the the record opentime was updated by the user. Hence we extract the original open time by sorting cases with similar case number in ascending order and taking the opentime from the record with earliest system updated time)
incident_processed['number'][incident_processed['lifetime'] < 0], incident_processed['opened_at'][incident_processed['lifetime'] < 0]

(489       INC0000174
 490       INC0000174
 491       INC0000174
 492       INC0000174
 493       INC0000174
              ...    
 141665    INC0118952
 141666    INC0118952
 141667    INC0118999
 141668    INC0118999
 141669    INC0118999
 Name: number, Length: 50482, dtype: object,
 489      2016-02-29 10:39:00
 490      2016-02-29 10:39:00
 491      2016-02-29 10:39:00
 492      2016-02-29 10:39:00
 493      2016-02-29 10:39:00
                  ...        
 141665   2017-10-02 14:48:00
 141666   2017-10-02 14:48:00
 141667   2017-10-02 15:55:00
 141668   2017-10-02 15:55:00
 141669   2017-10-02 15:55:00
 Name: opened_at, Length: 50482, dtype: datetime64[ns])

In [328]:
def update_same_case_records(df):
    df.reset_index()
    unique_case_ids = df['number'].unique()
    if len(unique_case_ids) > 0:
        for case_id in tqdm(unique_case_ids):
            #indicidual case id reccords
            df_case = df[df['number'] == case_id]
            #sort the column by system updated at
            sorted_df_case = df_case.sort_values('sys_updated_at')
            #sorted_df_case.reset_index(inplace=True)
            initial_record = sorted_df_case.iloc[[0]]
            latest_record = sorted_df_case.iloc[[sorted_df_case.shape[0]-1]]
            lifetime = latest_record['closed_at'] - latest_record['opened_at']
            lifetime = lifetime.dt.days.values[0]
            if  lifetime <= 0: latest_record['opened_at'] = initial_record['sys_updated_at'].values
            df = df[df['number'] != case_id]
            if df['number'].isnull().values.any():
                df = pd.concat([df,latest_record.to_frame()])
            else: df = pd.concat([df,latest_record])
    return df


In [329]:
df_same_case = incident_processed[incident_processed['lifetime'] < 0]
df_unique_case = incident_processed[incident_processed['lifetime'] > 0]

In [330]:
df_same_case = update_same_case_records(df_same_case)

100%|██████████| 8614/8614 [03:06<00:00, 46.28it/s]


In [341]:
#merge unique cases and the newly created df having single record for every case number
df_new = pd.concat([df_unique_case,df_same_case])

In [342]:
# create features lifetime and action_at
df_new['lifetime'] = df_new['closed_at'] - df_new['opened_at']
df_new['action_at'] = df_new['sys_updated_at'] - df_new['opened_at']

#store only days values as features
df_new['lifetime'] = df_new['lifetime'].dt.days
df_new['action_at'] = df_new['action_at'].dt.days

#scale lifetime and action_at features
features = ['lifetime','action_at']
df_new[features] = scaler.fit_transform(df_new[features])

In [343]:
df_new.head()

,number,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,caller_id,opened_by,opened_at,...,assigned_to,knowledge,u_priority_confirmation,notify,closed_code,resolved_by,resolved_at,closed_at,lifetime,action_at
0,INC0000045,3,1,0,0,0,1,1462,200,2016-02-29 01:16:00,...,73,1,0,0,12,51,2016-02-29 11:29:00,2016-05-03 12:00:00,0.106136,0.335171
1,INC0000045,4,1,0,0,2,1,1462,200,2016-02-29 01:16:00,...,73,1,0,0,12,51,2016-02-29 11:29:00,2016-05-03 12:00:00,0.106136,0.335171
2,INC0000045,4,1,0,0,3,1,1462,200,2016-02-29 01:16:00,...,73,1,0,0,12,51,2016-02-29 11:29:00,2016-05-03 12:00:00,0.106136,0.335171
3,INC0000045,2,0,0,0,4,1,1462,200,2016-02-29 01:16:00,...,73,1,0,0,12,51,2016-02-29 11:29:00,2016-05-03 12:00:00,0.106136,0.405733
4,INC0000047,3,1,0,0,0,1,1462,120,2016-02-29 04:40:00,...,223,1,0,0,12,196,2016-01-03 09:52:00,2016-06-03 10:00:00,0.157546,0.335171


In [344]:
#check if there are records with negative lifetime
df_new['number'][df_new['lifetime'] < 0], df_new['opened_at'][df_new['lifetime'] < 0]

(Series([], Name: number, dtype: object),
 Series([], Name: opened_at, dtype: datetime64[ns]))

In [345]:
# store case numbers for analytical purpose
ids = df_new['number']
df_new.drop(['number'],inplace = True, axis = 1)

In [346]:
df_new.columns

Index(['incident_state', 'active', 'reassignment_count', 'reopen_count',
       'sys_mod_count', 'made_sla', 'caller_id', 'opened_by', 'opened_at',
       'sys_created_by', 'sys_created_at', 'sys_updated_by', 'sys_updated_at',
       'contact_type', 'location', 'category', 'subcategory', 'u_symptom',
       'impact', 'urgency', 'priority', 'assignment_group', 'assigned_to',
       'knowledge', 'u_priority_confirmation', 'notify', 'closed_code',
       'resolved_by', 'resolved_at', 'closed_at', 'lifetime', 'action_at'],
      dtype='object')

In [347]:
# drop irrelevant features
df_new.drop(['closed_at','opened_at','sys_updated_at','sys_created_at','resolved_at','closed_code','category','subcategory','u_symptom','contact_type','caller_id','opened_by','sys_created_by','sys_updated_by','assigned_to','resolved_by','location','assignment_group'],axis = 1,inplace=True)

In [348]:
# reset index values and drop index column for the model input
df_new.reset_index(inplace= True)
df_new.drop(['index'],axis = 1,inplace=True)

In [349]:
df_new.head()

,incident_state,active,reassignment_count,reopen_count,sys_mod_count,made_sla,impact,urgency,priority,knowledge,u_priority_confirmation,notify,lifetime,action_at
0,3,1,0,0,0,1,1,1,2,1,0,0,0.106136,0.335171
1,4,1,0,0,2,1,1,1,2,1,0,0,0.106136,0.335171
2,4,1,0,0,3,1,1,1,2,1,0,0,0.106136,0.335171
3,2,0,0,0,4,1,1,1,2,1,0,0,0.106136,0.405733
4,3,1,0,0,0,1,1,1,2,1,0,0,0.157546,0.335171


In [350]:
df_new.to_csv('Data/reopened.csv')